# Rice Business Stock Market Data Portal - Getting Started

This notebook demonstrates how to connect to and query the Rice Business Stock Market Data Portal.

## Setup

1. Make sure you have your Rice Data Portal access token (JWT from your email)
2. Add your access token to the `.env` file in this folder
3. Run the cells to start exploring the data!

## Import and Connect

In [ ]:
from rice_data_client import RiceDataClient
from dotenv import load_dotenv
import pandas as pd
import os

# Load environment variables from .env file
load_dotenv()

# Get configuration from environment
ACCESS_TOKEN = os.getenv('USER_ACCESS_TOKEN')
BASE_URL = os.getenv('RICE_DATA_URL', 'https://portal.rice-business.org')

# Connect to Rice Data Portal
client = RiceDataClient(
    access_token=ACCESS_TOKEN,
    base_url=BASE_URL
)

## Example 1: Search for Tickers

Find technology companies using the convenience method:

In [ ]:
# Search for technology companies
tech_stocks = client.search_tickers(sector="Technology", limit=10)
tech_stocks

## Example 2: Get Ticker Details

Get detailed information for a specific company:

In [ ]:
# Get details for Apple
aapl_info = client.get_ticker_details("AAPL")
aapl_info

## Example 3: Get Price Data

Fetch historical price data from the SEP table:

In [ ]:
# Get recent price data for Apple
prices = client.get_price_data(
    ticker="AAPL",
    start_date="2024-01-01",
    limit=50
)
prices.head(10)

## Example 4: Custom SQL Query

Write your own SQL queries using DuckDB syntax:

In [ ]:
# Query for largest companies by market cap
sql = """
SELECT 
    d.ticker, 
    t.name,
    d.marketcap,
    d.date
FROM daily d
JOIN tickers t ON d.ticker = t.ticker
WHERE d.date = (SELECT MAX(date) FROM daily)
ORDER BY d.marketcap DESC
LIMIT 10
"""

largest_companies = client.query(sql)
largest_companies

## Example 5: Get Fundamental Data

Access financial statement data from the SF1 table:

In [ ]:
# Get annual financial data for Apple
fundamentals = client.get_fundamentals(
    ticker="AAPL",
    dimension="MRY",  # Most Recent, Annual
    limit=5
)
fundamentals

## Example 6: Calculate Financial Metrics

Query and calculate custom metrics:

In [ ]:
# Get quarterly revenue growth for Apple
sql = """
SELECT 
    ticker,
    reportperiod,
    revenue,
    LAG(revenue, 4) OVER (PARTITION BY ticker ORDER BY reportperiod) as revenue_year_ago,
    ROUND(
        ((revenue - LAG(revenue, 4) OVER (PARTITION BY ticker ORDER BY reportperiod)) / 
         LAG(revenue, 4) OVER (PARTITION BY ticker ORDER BY reportperiod)) * 100, 2
    ) as yoy_growth_pct
FROM sf1
WHERE ticker = 'AAPL'
  AND dimension = 'MRQ'
  AND reportperiod::DATE >= CURRENT_DATE - INTERVAL '3 years'
ORDER BY reportperiod DESC
LIMIT 12
"""

revenue_growth = client.query(sql)
revenue_growth

## Example 7: Visualize Data

Create simple visualizations with matplotlib:

In [ ]:
import matplotlib.pyplot as plt

# Get monthly closing prices
sql = """
SELECT 
    date,
    closeadj as price
FROM sep
WHERE ticker = 'AAPL'
  AND date::DATE >= CURRENT_DATE - INTERVAL '1 year'
ORDER BY date
"""

price_data = client.query(sql)
price_data['date'] = pd.to_datetime(price_data['date'])

# Plot
plt.figure(figsize=(12, 6))
plt.plot(price_data['date'], price_data['price'])
plt.title('AAPL Stock Price - Last Year')
plt.xlabel('Date')
plt.ylabel('Adjusted Close Price ($)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Next Steps

Now you're ready to explore the data! Here are some ideas:

1. **Explore sectors**: Use `client.list_sectors()` to see all available sectors
2. **Compare companies**: Query multiple tickers and compare metrics
3. **Time series analysis**: Look at trends over time using date filters
4. **Financial ratios**: Calculate ROE, profit margins, debt ratios, etc.
5. **Screen stocks**: Find companies meeting specific criteria

### Important Notes:

- All date columns are VARCHAR and must be cast to DATE: `date::DATE`
- Market cap values in DAILY table are in thousands of dollars
- SF1 dimension codes: MRY (annual), MRQ (quarterly), MRT (trailing 4Q)
- Use `reportperiod` as the main date field in SF1 table

For complete documentation, visit: https://portal-guide.rice-business.org